# First example with real images

The goal of this exercice is to train an artificial neural network on images of **handwritten digits** and check that the trained network is able to identify and classify new images that it did not see during the training.
We will use the **MNIST** database, which contains 60.000 images (28x28 pixels) for training and 10.000 for testing. 

This notebook was inspired from the EMBL notebooks from the Kreshuk lab (https://github.com/kreshuklab/teaching-dl-course-2020/).

## I - Downloading the MNIST image database

First step - we will import the MNIST database. 

In [ ]:
from tensorflow import keras
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Below, a set of 3 randomly selected images is displayed. For each image, the associated **label** (its real class, the ground truth) is indicated on the top. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import random

plt.rcParams['figure.figsize'] = (9,9)
for i in range(3):
    plt.subplot(1,3,i+1)
    num = random.randint(0, len(train_images))
    plt.imshow(train_images[num], cmap='gray', interpolation='none')
    plt.title("Class {}".format(train_labels[num]))
    
plt.tight_layout()

And the size of the images is indicated :

In [ ]:
im = train_images[0]
print('The size of the image is {} pixels'.format(im.shape))
print('The minimum pixel value is {} and the maxium  {}'.format(np.min(im), np.max(im)))

## II - Formatting the data

For the moment, we do not know how to work with 2D images. 
In the MNIST database, each image is composed of 28x28 pixels. The intensity is encoded in 8-bit, meaning that the intensity value of each pixel will be between 0 and 255=2⁸-1. 
For the input layer of the network, we need to transform those **images** into a **single-column tensor composed of 28x28=784 elements**. Each image is going to be **reshaped** or **flatten**. 

The data are also **normalized** so that all the values will be between 0 and 1.

In [ ]:
# the images are flatten into one single tensor containing 784 elements. The 
# images are also converted to float to allow for the normalization (else,
# if we keep the integer formation, the result of the normalization will be
# either 0 or 1)
# -------------

n_pixel = np.prod(im.shape)
X_train = train_images.reshape(train_images.shape[0], n_pixel).astype('float32')
X_val = test_images.reshape(test_images.shape[0], n_pixel).astype('float32')

# normalize inputs from 0-255 to 0-1
# ----------------------------------

X_train /= 255
X_val /= 255

The labels are also changed from **single digit** to **categorical or one-hot format**. Example:

class "0" : [**1** 0 0 0 0 0 0 0 0 0]

class "1" : [0 **1** 0 0 0 0 0 0 0 0]

etc..

In [ ]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(train_labels)
Y_val = np_utils.to_categorical(test_labels)

for n in range(5):
  print("The previous label was " + str(train_labels[n]) + " and is now " + str(Y_train[n]))

## III- Creating and training a simple network for digit classification

We will now build a simple network able to read the modify images as input and return a vector (in the "one-hot" format) indicating the predicted class for the image. 
For now, the structure of this network remains very similar to what we have already seen yesterday. 

In [ ]:
# Import the keras libraries 
# --------------------------
from ... import ...

# Define the architecture of the network. Note that the activation function for 
# the last layer is "softmax", which gives the probability for each of the 10 classes
# ---------------------------

model = ...

# Compile the model defining the optimizer and the loss function 
# --------------------------------------------------------------
model.compile(...)

# Return a full description of the network
# ----------------------------------------
model.summary()

(Optional) 

In order to find the best network architecture for your problem, many training will be performed and compared. TensorFlow offers several vizualization tools to facilitate this task. 

TensorBoard provides the visualization and tooling needed for machine learning experimentation:
*   Tracking and visualizing metrics such as loss and accuracy
*   Visualizing the model graph (ops and layers)
*   Viewing histograms of weights, biases, or other tensors as they change over time
*   Projecting embeddings to a lower dimensional space
*   Displaying images, text, and audio data
*   Profiling TensorFlow programs
*   And much more

After running the following block, you should click on the generated link.

In [ ]:
from keras.callbacks import TensorBoard  #Visulization of Accuracy and loss

# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%load_ext tensorboard

tb_FC = TensorBoard('runs/MNIST_dense_model', histogram_freq=1)
%tensorboard --logdir runs

Finallly, we are going to train the model (with `model.fit()` ) and follow in real-time how the network is able to learn. We send for training a batch of 32 images each iteration, until all traing images are used. We then repeat this 50 times (i.e. 50 epochs). Look each epoch how the loss decreases and the accuracy increases, both on the training and validation (`val_*`) images. The best we aim for is `val_accuracy = 1` (but >0.9 is already not bad).

In [ ]:
history = model.fit(X_train, Y_train, 
          validation_data=(X_val, Y_val),
          epochs=50, batch_size=32,
          verbose=1,
          callbacks=[tb_FC])

## IV- Evaluate the model accuracy :

After training is done, you can calculate the accuracy of the model using the test set. You feed the trained model with all the test images (which have their respective labels known), and see how good are the predictions of the model, i.e. how often it predicts the right class. It will be likely not too different from the `val_accuracy` obtained at the end of the training.

In [ ]:
test_loss, test_acc = model.evaluate(X_val, Y_val)
print('test_acc:', test_acc)

It is always useful to check the performance of the network on randomly selected images. More precisely, to improve our network, we need to understand why the network is sometimes failing. 
Below, we are going to test the network on the validation set (i.e. images not seen during training) and select images for which the network prediction is right and other for which the predition is wrong.  

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
# -----------------------------------------------------------

predicted_classes = model.predict(X_val)
predicted_classes = np.argmax(predicted_classes, axis=1)

# Check which items we got right / wrong
# --------------------------------------

correct_indices = np.nonzero(predicted_classes == test_labels)[0]

incorrect_indices = np.nonzero(predicted_classes != test_labels)[0]

print(len(correct_indices))
print(len(incorrect_indices))

Display a few example for which the network is right

In [ ]:
plt.figure()
for i, correct in enumerate(correct_indices[:6]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_val[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], test_labels[correct]))
    
plt.tight_layout()

... and a few where the network is wrong!

In [ ]:
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:6]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_val[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], test_labels[incorrect]))
    
plt.tight_layout()

And finally the accuracy and loss along the training epochs can be plotted :

In [ ]:
history_dict = history.history

# Plot the evolution of the accuracy during the training
# ------------------------------------------------------

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']

n = len(acc_values)
epochs = range(1, n+1)

plt.subplot(2,1,1)
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.legend()
plt.show()

# Plot the evolution of the loss during the training
# ------------------------------------------------------

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

plt.subplot(2,1,2)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Loss', fontsize=15)
plt.legend()
plt.show()

## V- Introduction to CNN = Convolutional Neural Network

Let's move to the next step, improving the model performace: convolutional nets.

In order to work with densely connected network, the images need to be reshape and flatten. **Useful spatial information is lost in this process**. 
**Convolutional network** has been introduced to work with 2D/3D images and analyze the spatial context around the pixels. Such network are able to **learn which features of your image (curve, intensity, shape, etc.) are important for the classficiation**. 
The network will learn **"filters"** that will be applied to each image in order to highlight specific features of your images.   



To start we need to reload the MNIST database and import new libraries specific to CNN.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import random

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential # model type to be used
from keras import Model # used for the visualization of the features maps
from keras.utils import np_utils
from keras.datasets import mnist
from keras.callbacks import TensorBoard  #Visulization of Accuracy and loss

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

A normalization step is again required, though this time we are keeping the shape of the image (28x28 pixels). 
The labels are also changed again to the one-hot format.

In [ ]:
X_train = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
X_val = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')

# normalize inputs from 0-255 to 0-1
# ----------------------------------

X_train /= 255
X_val /= 255

# change the label from single digit to one-hot format
# ----------------------------------------------------

Y_train = np_utils.to_categorical(train_labels)
Y_val = np_utils.to_categorical(test_labels)

Now, we are creating a simple CNN to analyze the images!


In [ ]:
modelCNN = Sequential([
    
    # Convolution Layer 1
    Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)), # 16 different 3x3 kernels -- so 16 feature maps
    MaxPooling2D(pool_size=(2, 2)), # Pool the max values over a 2x2 kernel

    # Convolution Layer 2
    Conv2D(16, (3, 3), activation='relu'), # 16 different 3x3 kernels 
    MaxPooling2D(pool_size=(2, 2)),

    # Convolution Layer 3
    Conv2D(16, (3, 3), activation='relu'), # 16 different 3x3 kernels

    Flatten(), # Flatten final 3x3x16 output matrix into a 144-length vector 

    # Fully Connected Layer 4
    Dense(15), # 15 FCN nodes
    Activation('relu'),
    Dense(10), # Necessary for the last layer since we have 10 classes
    Activation('softmax'),
])
modelCNN.summary()

And finally we are training this network :

In [ ]:
%load_ext tensorboard

tb_CNN = TensorBoard('runs/CNN_model', histogram_freq=1)
%tensorboard --logdir runs

modelCNN.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

history = modelCNN.fit(X_train, Y_train, 
          validation_data=(X_val, Y_val),
          epochs=10, batch_size=32,
          verbose=1,
          callbacks=[tb_CNN])

In [ ]:
history_dict = history.history

# Plot the evolution of the accuracy during the training
# ------------------------------------------------------

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']

n = len(acc_values)
epochs = range(1, n+1)

plt.rcParams['figure.figsize'] = (9,9) # Make the figures a bit bigger

plt.subplot(2,1,1)
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.legend()
plt.show()

# Plot the evolution of the loss during the training
# ------------------------------------------------------

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

plt.subplot(2,1,2)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Loss', fontsize=15)
plt.legend()
plt.show()

##VI- Kernel and features map 

Finally we can visualize what the network is learning. Below, a simple script is used to display the filters that are learnt by the network as well as the feature maps. 

For more details regarding the code you can go [here](https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/) and for a user-friendly visualization app [here](https://www.cs.ryerson.ca/~aharley/vis/conv/).

In [ ]:
# Indicate the layer you wish to observe (works only for the conv layers)
n_layer = 0

# Retrieve the weights from the chosen layer
filters, biases = modelCNN.layers[n_layer].get_weights()
print('The shape of filters is {}'.format(filters.shape))
	
# normalize filter values between 0-1
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

# plot the 16 filters from the convolutional layer

for i in range(16):
	# get the filter
	f = filters[:, :, :, i]

	# plot each filter in gray scale

	ax = plt.subplot(4, 4, i+1)
	ax.set_xticks([])
	ax.set_yticks([])
	plt.imshow(f[:,:,0], cmap='gray')
 
# show the figure
plt.show()

In [ ]:
# Select one image from the validation set
n_im = 0

im = X_val[n_im]
im = np.expand_dims(im, axis=0) 

# get feature maps by redefining the model to output only the first hidden layer
n_layer = 1
model = Model(inputs=modelCNN.inputs, outputs=modelCNN.layers[n_layer].output)

feature_maps = model.predict(im)
print("The size of features map is {}. There are as many maps as the number of filters in the convolution layer.".format(feature_maps.shape))
	
# plot all 16 maps

for i in range(16):
	ax = plt.subplot(4, 4, i+1)
	ax.set_xticks([])
	ax.set_yticks([])
	# plot filter channel in grayscale
	plt.imshow(feature_maps[0, :, :, i], cmap='gray')

# show the figure
plt.show()
